# Monte Carlo Algorithms - Experiments - Chapter 5 and 6.1

In [ ]:
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt

from typing import List, Optional
from shapely.geometry import Polygon, MultiPolygon, Point

from experimental.utils.dynamic_system import DynamicSystem
from experimental.utils.markov_decision_process import MarkovDecisionProcess

## Define System Dynamics

In [ ]:
A = np.array([[1,1], [1,0]])

In [ ]:
def phi(x: np.array) -> np.array:
    x_new = np.dot(A, x) % 1
    return x_new

In [ ]:
def q(x: np.array) -> np.array:
    return x % 1

In [ ]:
# get eigenvalues and right eigenvectors, i.e. transposed eigenvectors
eig_vals, eig_vects = np.linalg.eig(A)
# retrieve eigenvectors from right eigenvectors
eig_vects = np.transpose(eig_vects)

In [ ]:
print(f"Eigenvalues: {eig_vals}")

In [ ]:
print(f"Eigenvectors: {eig_vects}")

In [ ]:
v1 = eig_vects[0]
v2 = eig_vects[1]

## Calculate Inetrsection Points between Lines

In [ ]:
def perp(a) :
    b = np.empty_like(a)
    b[0] = -a[1]
    b[1] = a[0]
    return b

# line segment a given by endpoints a1, a2
# line segment b given by endpoints b1, b2
def seg_intersect(a, b) :
    a1, a2 = a
    b1, b2 = b

    da = a2 - a1
    db = b2 - b1
    dp = a1 - b1
    dap = perp(da)
    denom = np.dot(dap, db)
    num = np.dot(dap, dp)

    return (num / denom.astype(float))*db + b1

## Build Markov Partition

In [ ]:
u1 = np.array([0,0])
u2 = np.array([0,1])
u3 = np.array([1,1])
u4 = np.array([1,0])

In [ ]:
lu1 = np.array([[0,0], [0,1]])
lu2 = np.array([[0,1], [1,1]])
lu3 = np.array([[1,1], [1,0]])
lu4 = np.array([[1,0], [0,0]])

In [ ]:
p1 = u1 + (1/v1[0]) * v1
p2 = u2 - (1/v2[1]) * v2
p3 = u3 - (1/v1[0]) * v1
p4 = u4 + (1/v2[1]) * v2

In [ ]:
l_00 = np.array([u1, p1])
l_01 = np.array([u2, p2])
l_10 = np.array([u4, p4])
l_11 = np.array([u3, p3])

In [ ]:
P1a = seg_intersect(l_00, l_01)
P1b = seg_intersect(l_10, l_00)

P3a = seg_intersect(l_10, l_11)

symm_helper_x = seg_intersect(lu2, l_10)
symm_helper_y = symm_helper_x - np.array([0,1])
symm_helper_l11 = np.array([symm_helper_x, symm_helper_y])

p1_symm = symm_helper_y + v2
l_10_symm_extension = np.array([symm_helper_y, p1_symm])
P3b = seg_intersect(l_10_symm_extension, l_00)
l_10_symm_extension = np.array([symm_helper_y, P3b])

In [ ]:
l_00 = np.array([u1, P1b])
l_01 = np.array([u2, P1a])
l_10 = np.array([u4, p4])
l_11 = np.array([u3, P3a])

In [ ]:
# plot unit square
plt.plot(lu1[:, 0], lu1[:, 1], "r-")
plt.plot(lu2[:, 0], lu2[:, 1], "r-")
plt.plot(lu3[:, 0], lu3[:, 1], "r-")
plt.plot(lu4[:, 0], lu4[:, 1], "r-")

# step 1: plot l_10 in contracting direction
plt.plot(l_10[:, 0], l_10[:, 1], "b-")

# step 2: plot l_00 and l_11 in expanding directions
plt.plot(l_00[:, 0], l_00[:, 1], "b-")
plt.plot(l_11[:, 0], l_11[:, 1], "b-")

# step 3: plot l_01 in contracting direction 
plt.plot(l_01[:, 0], l_01[:, 1], "b-")

# step 4: plot symmetric extension of l_01 line
plt.plot(symm_helper_l11[:, 0], symm_helper_l11[:, 1], "m--")
plt.plot(l_10_symm_extension[:, 0], l_10_symm_extension[:, 1], "b-")

# plot intersection points
plt.plot(P1a[0], P1a[1], "bo")
plt.plot(P1b[0], P1b[1], "ro")
plt.plot(P3a[0], P3a[1], "go")
plt.plot(P3b[0], P3b[1], "yo")

plt.show()

## Transform Partition to Shapely-Object

In [ ]:
P1A = Polygon([P1a, u1, u2])
P1B = Polygon([P3a, u3, u4])
P1 = MultiPolygon([P1A, P1B])

In [ ]:
P2A = Polygon([P3b, u1, symm_helper_l11[1]])
P2B = Polygon([symm_helper_l11[0], u2, P1a, P1b])
P2 = MultiPolygon([P2A, P2B])

In [ ]:
P3A = Polygon([P1b, u4, symm_helper_l11[1], P3b])
P3B = Polygon([symm_helper_l11[0], u3, P3a])
P3 = MultiPolygon([P3A, P3B])

In [ ]:
partition = [P1, P2, P3]

In [ ]:
MultiPolygon([P1A, P1B, P2A, P2B, P3A, P3B])

In [ ]:
P1A_phi = Polygon([P1b, u4, u1])
P1B_phi = Polygon([u2, u3, phi(P3a)])
P1_phi = MultiPolygon([P1A_phi, P1B_phi])

In [ ]:
symm_helper_x = seg_intersect(lu3, l_00)
symm_helper_y = q(symm_helper_x)
symm_helper_l00 = np.array([symm_helper_x, symm_helper_y])

In [ ]:
P2A_phi = Polygon([symm_helper_l00[1], phi(P1b), P1a, u1])
P2B_phi = Polygon([symm_helper_l00[0], u4, P1b])
P2_phi = MultiPolygon([P2A_phi, P2B_phi])

In [ ]:
P3A_phi = Polygon([phi(P3a), P1a, symm_helper_l00[0], u3])
P3B_phi = Polygon([u2, phi(P1b), symm_helper_l00[1]])
P3_phi = MultiPolygon([P3A_phi, P3B_phi])

In [ ]:
MultiPolygon([P1A_phi, P1B_phi, P2A_phi, P2B_phi, P3A_phi, P3B_phi])

In [ ]:
partition_phi = [P1_phi, P2_phi, P3_phi]

## Try Monte Carlo Algorithms

In [ ]:
def phi_non_identified(x: np.array) -> np.array:
    return np.dot(A, x)

In [ ]:
def d_phi(x: np.array) -> np.array:
    return np.transpose(A)

In [ ]:
m_id = 1
dynamic_system = DynamicSystem(phi_non_identified, d_phi, m_id)
markov_decision_process = MarkovDecisionProcess(dynamic_system, partition)

In [ ]:
markov_decision_process.estimate_probability_matrix_pi_method(c=1000, tau=0.001, max_sample_trials=1000)

In [ ]:
markov_decision_process.estimate_probability_matrix_random_walker_method(l=50, m=1000, max_sample_trials=1000)

## Compute Probability Ground truth

In [ ]:
prob_mat_ground_truth = markov_decision_process.ground_truth_probability_matrix(partition_phi)

In [ ]:
prob_mat_ground_truth

## Experiments - Monte Carlo Algorithm Random Walk Method

In [ ]:
m_candidates = list(np.ceil(np.linspace(start=1, stop=300, num=20)))
l_candidates = list(np.ceil(np.linspace(start=1, stop=100, num=20)))
repetitions = 10
compute_experiments = False

In [ ]:
if compute_experiments:
    estimation_error = {}
    for m_candidate in m_candidates:
        estimation_error[m_candidate] = {}
        for l_candidate in l_candidates:
            estimation_error[m_candidate][l_candidate] = {}
            for i in range(repetitions):
                rep_key = f"rep_{i+1}"
                estimation_error[m_candidate][l_candidate][rep_key] = {}
                print(f"m={m_candidate}, l={l_candidate}, Rep: {i+1}")

                start_time = time.time()
                prob_mat_estimated = markov_decision_process.estimate_probability_matrix_random_walker_method(l=l_candidate, m=m_candidate, max_sample_trials=1000)
                end_time = time.time()
                run_time = round(end_time - start_time, 2)

                estimation_error[m_candidate][l_candidate][rep_key]["mean"] = np.mean(np.abs(prob_mat_estimated - prob_mat_ground_truth))
                estimation_error[m_candidate][l_candidate][rep_key]["sum"] = np.sum(np.abs(prob_mat_estimated - prob_mat_ground_truth))
                estimation_error[m_candidate][l_candidate][rep_key]["median"] = np.median(np.abs(prob_mat_estimated - prob_mat_ground_truth))
                estimation_error[m_candidate][l_candidate][rep_key]["max"] = np.max(np.abs(prob_mat_estimated - prob_mat_ground_truth))
                estimation_error[m_candidate][l_candidate][rep_key]["run_time"] = run_time

                print(f"Estimation error measures: {estimation_error[m_candidate][l_candidate][rep_key]} \n")

    with open(f"results/estimate_prob_random_walk_method.pkl", "wb") as file:
        pickle.dump(estimation_error, file, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f"results/estimate_prob_random_walk_method.pkl", "rb") as file:
        estimation_error = pickle.load(file)

In [ ]:
estimation_error_averaged = {}
for m_candidate in m_candidates:
    estimation_error_averaged[m_candidate] = {}
    for l_candidate in l_candidates:
        estimation_error_averaged[m_candidate][l_candidate] = {}

        means = np.zeros(repetitions)
        sums = np.zeros(repetitions)
        medians = np.zeros(repetitions)
        maxs = np.zeros(repetitions)
        run_times = np.zeros(repetitions)

        for i in range(repetitions):
            rep_key = f"rep_{i+1}"
            means[i] = estimation_error[m_candidate][l_candidate][rep_key]["mean"]
            sums[i] = estimation_error[m_candidate][l_candidate][rep_key]["sum"]
            medians[i] = estimation_error[m_candidate][l_candidate][rep_key]["median"]
            maxs[i] = estimation_error[m_candidate][l_candidate][rep_key]["max"]
            run_times[i] = estimation_error[m_candidate][l_candidate][rep_key]["run_time"]

        estimation_error_averaged[m_candidate][l_candidate]["mean"] = {"avg": np.mean(means), "var": np.var(means)}
        estimation_error_averaged[m_candidate][l_candidate]["sum"] = {"avg": np.mean(sums), "var": np.var(sums)}
        estimation_error_averaged[m_candidate][l_candidate]["median"] = {"avg": np.mean(medians), "var": np.var(medians)}
        estimation_error_averaged[m_candidate][l_candidate]["max"] = {"avg": np.mean(maxs), "var": np.var(maxs)}
        estimation_error_averaged[m_candidate][l_candidate]["run_time"] = {"avg": np.mean(run_times), "var": np.var(run_times)}

In [ ]:
small_l = 1
medium_l = 12
large_l = 54

small_tau_m_mean_errors = {}
medium_tau_m_mean_errors = {}
large_tau_m_mean_errors = {}

small_tau_m_sum_errors = {}
medium_tau_m_sum_errors = {}
large_tau_m_sum_errors = {}

small_tau_m_max_errors = {}
medium_tau_m_max_errors = {}
large_tau_m_max_errors = {}

small_tau_m_run_times = {}
medium_tau_m_run_times = {}
large_tau_m_run_times = {}

for m in m_candidates:
    small_tau_m_mean_errors[m] = estimation_error_averaged[m][small_l]["mean"]["avg"]
    medium_tau_m_mean_errors[m] = estimation_error_averaged[m][medium_l]["mean"]["avg"]
    large_tau_m_mean_errors[m] = estimation_error_averaged[m][large_l]["mean"]["avg"]

    small_tau_m_sum_errors[m] = estimation_error_averaged[m][small_l]["sum"]["avg"]
    medium_tau_m_sum_errors[m] = estimation_error_averaged[m][medium_l]["sum"]["avg"]
    large_tau_m_sum_errors[m] = estimation_error_averaged[m][large_l]["sum"]["avg"]

    small_tau_m_max_errors[m] = estimation_error_averaged[m][small_l]["max"]["avg"]
    medium_tau_m_max_errors[m] = estimation_error_averaged[m][medium_l]["max"]["avg"]
    large_tau_m_max_errors[m] = estimation_error_averaged[m][large_l]["max"]["avg"]

    small_tau_m_run_times[m] = estimation_error_averaged[m][small_l]["run_time"]["avg"]
    medium_tau_m_run_times[m] = estimation_error_averaged[m][medium_l]["run_time"]["avg"]
    large_tau_m_run_times[m] = estimation_error_averaged[m][large_l]["run_time"]["avg"]

In [ ]:
plt.plot(small_tau_m_mean_errors.keys(), small_tau_m_mean_errors.values(), color="blue", label=f"l = {small_l}")
plt.plot(medium_tau_m_mean_errors.keys(), medium_tau_m_mean_errors.values(), color="green", label=f"l = {medium_l}")
plt.plot(large_tau_m_mean_errors.keys(), large_tau_m_mean_errors.values(), color="red", label=f"l = {large_l}")

plt.legend(loc="upper right")
plt.xlabel("Candidates parameter m")
plt.ylabel("MEAN_ERROR")
plt.title("Mean error per state transistion probability estimate")

plt.savefig("results/plots/mean_error_random_walker.png", dpi=600)
plt.show()

In [ ]:
plt.plot(small_tau_m_sum_errors.keys(), small_tau_m_sum_errors.values(), color="blue", label=f"l = {small_l}")
plt.plot(medium_tau_m_sum_errors.keys(), medium_tau_m_sum_errors.values(), color="green", label=f"l = {medium_l}")
plt.plot(large_tau_m_sum_errors.keys(), large_tau_m_sum_errors.values(), color="red", label=f"l = {large_l}")

plt.legend(loc="upper right")
plt.xlabel("Candidates parameter m")
plt.ylabel("SUM_ERROR")
plt.title("Sum of estimation errors over all transitions")

plt.savefig("results/plots/sum_error_random_walker.png", dpi=600)
plt.show()

In [ ]:
plt.plot(small_tau_m_max_errors.keys(), small_tau_m_max_errors.values(), color="blue", label=f"l = {small_l}")
plt.plot(medium_tau_m_max_errors.keys(), medium_tau_m_max_errors.values(), color="green", label=f"l = {medium_l}")
plt.plot(large_tau_m_max_errors.keys(), large_tau_m_max_errors.values(), color="red", label=f"l = {large_l}")

plt.legend(loc="upper right")
plt.xlabel("Candidates parameter m")
plt.ylabel("MAX_ERROR")
plt.title("Max of estimation errors over all transitions")

plt.savefig("results/plots/max_error_random_walker.png", dpi=600)
plt.show()

In [ ]:
plt.plot(small_tau_m_run_times.keys(), small_tau_m_run_times.values(), color="blue", label=f"l = {small_l}")
plt.plot(medium_tau_m_run_times.keys(), medium_tau_m_run_times.values(), color="green", label=f"l = {medium_l}")
plt.plot(large_tau_m_run_times.keys(), large_tau_m_run_times.values(), color="red", label=f"l = {large_l}")

plt.legend(loc="upper right")
plt.xlabel("Candidates parameter m")
plt.ylabel("Run time [s]")
plt.title("Run time dependence on m")

plt.savefig("results/plots/run_time_random_walker.png", dpi=600)
plt.show()

## Experiments - Monte Carlo Algorithm Pi Method

In [ ]:
c_candidates = list(np.ceil(np.linspace(start=1, stop=300, num=20)))
tau_candidates = list(1 / np.logspace(start=1, stop=5, num=10))
repetitions = 10
compute_experiments = False

In [ ]:
if compute_experiments:
    estimation_error = {}
    for c_candidate in c_candidates:
        estimation_error[c_candidate] = {}
        for tau_candidate in tau_candidates:
            estimation_error[c_candidate][tau_candidate] = {}
            for i in range(repetitions):
                rep_key = f"rep_{i+1}"
                estimation_error[c_candidate][tau_candidate][rep_key] = {}
                print(f"c={c_candidate}, tau={tau_candidate}, Rep: {i+1}")

                start_time = time.time()
                prob_mat_estimated, _ = markov_decision_process.estimate_probability_matrix_pi_method(c=c_candidate, tau=tau_candidate, max_sample_trials=1000)
                end_time = time.time()
                run_time = round(end_time - start_time, 2)

                estimation_error[c_candidate][tau_candidate][rep_key]["mean"] = np.mean(np.abs(prob_mat_estimated - prob_mat_ground_truth))
                estimation_error[c_candidate][tau_candidate][rep_key]["sum"] = np.sum(np.abs(prob_mat_estimated - prob_mat_ground_truth))
                estimation_error[c_candidate][tau_candidate][rep_key]["median"] = np.median(np.abs(prob_mat_estimated - prob_mat_ground_truth))
                estimation_error[c_candidate][tau_candidate][rep_key]["max"] = np.max(np.abs(prob_mat_estimated - prob_mat_ground_truth))
                estimation_error[c_candidate][tau_candidate][rep_key]["run_time"] = run_time

                print(f"Estimation error measures: {estimation_error[c_candidate][tau_candidate][rep_key]} \n")

    with open(f"results/estimate_prob_pi_method.pkl", "wb") as file:
        pickle.dump(estimation_error, file, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f"results/estimate_prob_pi_method.pkl", "rb") as file:
        estimation_error = pickle.load(file)

In [ ]:
estimation_error_averaged = {}
for c_candidate in c_candidates:
    estimation_error_averaged[c_candidate] = {}
    for tau_candidate in tau_candidates:
        estimation_error_averaged[c_candidate][tau_candidate] = {}

        means = np.zeros(repetitions)
        sums = np.zeros(repetitions)
        medians = np.zeros(repetitions)
        maxs = np.zeros(repetitions)
        run_times = np.zeros(repetitions)

        for i in range(repetitions):
            rep_key = f"rep_{i+1}"
            means[i] = estimation_error[c_candidate][tau_candidate][rep_key]["mean"]
            sums[i] = estimation_error[c_candidate][tau_candidate][rep_key]["sum"]
            medians[i] = estimation_error[c_candidate][tau_candidate][rep_key]["median"]
            maxs[i] = estimation_error[c_candidate][tau_candidate][rep_key]["max"]
            run_times[i] = estimation_error[c_candidate][tau_candidate][rep_key]["run_time"]

        estimation_error_averaged[c_candidate][tau_candidate]["mean"] = {"avg": np.mean(means), "var": np.var(means)}
        estimation_error_averaged[c_candidate][tau_candidate]["sum"] = {"avg": np.mean(sums), "var": np.var(sums)}
        estimation_error_averaged[c_candidate][tau_candidate]["median"] = {"avg": np.mean(medians), "var": np.var(medians)}
        estimation_error_averaged[c_candidate][tau_candidate]["max"] = {"avg": np.mean(maxs), "var": np.var(maxs)}
        estimation_error_averaged[c_candidate][tau_candidate]["run_time"] = {"avg": np.mean(run_times), "var": np.var(run_times)}

In [ ]:
small_tau = 1e-05
medium_tau = 0.001668100537200059
large_tau = 0.1

small_tau_c_mean_errors = {}
medium_tau_c_mean_errors = {}
large_tau_c_mean_errors = {}

small_tau_c_sum_errors = {}
medium_tau_c_sum_errors = {}
large_tau_c_sum_errors = {}

small_tau_c_max_errors = {}
medium_tau_c_max_errors = {}
large_tau_c_max_errors = {}

small_tau_c_run_times = {}
medium_tau_c_run_times = {}
large_tau_c_run_times = {}

for c in c_candidates:
    small_tau_c_mean_errors[c] = estimation_error_averaged[c][small_tau]["mean"]["avg"]
    medium_tau_c_mean_errors[c] = estimation_error_averaged[c][medium_tau]["mean"]["avg"]
    large_tau_c_mean_errors[c] = estimation_error_averaged[c][large_tau]["mean"]["avg"]

    small_tau_c_sum_errors[c] = estimation_error_averaged[c][small_tau]["sum"]["avg"]
    medium_tau_c_sum_errors[c] = estimation_error_averaged[c][medium_tau]["sum"]["avg"]
    large_tau_c_sum_errors[c] = estimation_error_averaged[c][large_tau]["sum"]["avg"]

    small_tau_c_max_errors[c] = estimation_error_averaged[c][small_tau]["max"]["avg"]
    medium_tau_c_max_errors[c] = estimation_error_averaged[c][medium_tau]["max"]["avg"]
    large_tau_c_max_errors[c] = estimation_error_averaged[c][large_tau]["max"]["avg"]

    small_tau_c_run_times[c] = estimation_error_averaged[c][small_tau]["run_time"]["avg"]
    medium_tau_c_run_times[c] = estimation_error_averaged[c][medium_tau]["run_time"]["avg"]
    large_tau_c_run_times[c] = estimation_error_averaged[c][large_tau]["run_time"]["avg"]

In [ ]:
plt.plot(small_tau_c_mean_errors.keys(), small_tau_c_mean_errors.values(), color="blue", label=f"tau = {small_tau}")
plt.plot(medium_tau_c_mean_errors.keys(), medium_tau_c_mean_errors.values(), color="green", label=f"tau = {round(medium_tau,3)}")
plt.plot(large_tau_c_mean_errors.keys(), large_tau_c_mean_errors.values(), color="red", label=f"tau = {large_tau}")

plt.legend(loc="upper right")
plt.xlabel("Candidates parameter c")
plt.ylabel("MEAN_ERROR")
plt.title("Mean error per state transistion probability estimate")

plt.savefig("results/plots/mean_error_pi_method.png", dpi=600)
plt.show()

In [ ]:
plt.plot(small_tau_c_sum_errors.keys(), small_tau_c_sum_errors.values(), color="blue", label=f"tau = {small_tau}")
plt.plot(medium_tau_c_sum_errors.keys(), medium_tau_c_sum_errors.values(), color="green", label=f"tau = {round(medium_tau,3)}")
plt.plot(large_tau_c_sum_errors.keys(), large_tau_c_sum_errors.values(), color="red", label=f"tau = {large_tau}")

plt.legend(loc="upper right")
plt.xlabel("Candidates parameter c")
plt.ylabel("SUM_ERROR")
plt.title("Sum of estimation errors over all transitions")

plt.savefig("results/plots/sum_error_pi_method.png", dpi=600)
plt.show()

In [ ]:
plt.plot(small_tau_c_max_errors.keys(), small_tau_c_max_errors.values(), color="blue", label=f"tau = {small_tau}")
plt.plot(medium_tau_c_max_errors.keys(), medium_tau_c_max_errors.values(), color="green", label=f"tau = {round(medium_tau,3)}")
plt.plot(large_tau_c_max_errors.keys(), large_tau_c_max_errors.values(), color="red", label=f"tau = {large_tau}")

plt.legend(loc="upper right")
plt.xlabel("Candidates parameter c")
plt.ylabel("MAX_ERROR")
plt.title("Max of estimation errors over all transitions")

plt.savefig("results/plots/max_error_pi_method.png", dpi=600)
plt.show()

In [ ]:
plt.plot(small_tau_c_run_times.keys(), small_tau_c_run_times.values(), color="blue", label="tau = 1e-5")
plt.plot(medium_tau_c_run_times.keys(), medium_tau_c_run_times.values(), color="green", label="tau = 1e-3")
plt.plot(large_tau_c_run_times.keys(), large_tau_c_run_times.values(), color="red", label="tau = 1e-1")


plt.legend(loc="upper right")
plt.xlabel("Candidates parameter c")
plt.ylabel("Run time [s]")
plt.title("Run time dependence on c")

plt.savefig("results/plots/run_time_pi_method.png",dpi=600)
plt.show()